# CNN в задаче регрессии

## Импорт библиотек

In [23]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision
from torchvision.transforms import v2

In [33]:
# выбор процессора

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Импорт датасета

In [34]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "_datasets")))

In [35]:
from BWSquares_ds import BWSquare_DS

In [36]:
bws_transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(dtype = torch.float32, scale=True),
        v2.Normalize(mean=(0.5,), std = (0.5,))
    ]
)  

In [37]:
BWSquares_ds_path = os.path.join(os.getcwd(), "..", "_datasets/BWSquares")

In [38]:
bws_dataset = BWSquare_DS(BWSquares_ds_path, transform=bws_transform)

In [39]:
bws_train_data, bws_val_data, bws_test_data = random_split(bws_dataset, [0.7, 0.1, 0.2])

In [40]:
# Разбиение на батчи

bws_train_loader = DataLoader(bws_train_data, batch_size=32, shuffle=True)
bws_val_loader = DataLoader(bws_val_data, batch_size=32, shuffle=False)
bws_test_loader = DataLoader(bws_test_data, batch_size=32, shuffle=False)

In [41]:
len(bws_dataset)

100000

## Создание модели CNN

In [42]:
class Regr_Model(nn.Module):
    def __init__(self, in_channels, out):
        super().__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 32, (3, 3), bias=False), # (batch_size, in_channels, 64, 64) => (batch_size, 32, 62, 62)
            nn.ReLU(),
            nn.Conv2d(32, 64, (3, 3), bias=False),          # (batch_size, 26, 62, 62) => (batch_size, 64, 60, 60)
            nn.ReLU()
        )

        self.flatten = nn.Flatten()                         # (batch_size, 64, 60, 60) => (batch_size, 64*60*60) 

        self.fc = nn.Sequential(
            nn.Linear(64*60*60, 128),                       # (batch_size, 128) 
            nn.ReLU(),
            nn.Linear(128, out)                             # (batch_size, out)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [43]:
# Проверка модели

model_regression = Regr_Model(1, 2).to(device)
model_regression

Regr_Model(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (3): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=230400, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [44]:
class EarlyStopping:
    def __init__(self, mode='min', patience=10, threshold=0.0001, threshold_mode='rel'):
        if mode not in {'min', 'max'}:
            raise ValueError(f"Параметр mode может принимать только значения max и min.")
        if threshold_mode not in {'rel', 'abs'}:
            raise ValueError(f"Параметр threshold_mode может принимать только значения rel и abs.")
        if not isinstance(patience, int):
            raise TypeError(f"Параметр patience должен быть целым числом.")
        if not isinstance(threshold, float):
            raise TypeError(f"Параметр threshold должен быть float.")
        if threshold >= 1.0:
            raise ValueError(f"Параметр threshold должен быть меньше 1,0.")
        
        self.mode = mode
        self.patience = patience
        self.threshold = threshold
        self.threshold_mode = threshold_mode
        self.count = 0
        self.best = None


    def __call__(self, tracked_parameter): # tracked_parameter - это либо ф-ция потерь, либо значение метрики
        current = float(tracked_parameter)
        
        if self.best is None:
            self.best = current
            return False
        
        if self.changed_better(current, self.best):
            self.best = current
            self.count = 0
        else:
            self.count += 1
        
        if self.count >= self.patience:
            return True  # сигнал к остановке
        return False  # продолжаем обучение


    def changed_better(self, current, best):
        if self.mode == 'min' and self.threshold_mode == 'rel':
            return current < best * (1 - self.threshold)
        elif self.mode == 'min' and self.threshold_mode == 'abs':
            return current < best - self.threshold
        elif self.mode == 'max' and self.threshold_mode == 'rel':
            return current > best * (1 + self.threshold)
        else:  # mode == 'max' and threshold_mode == 'abs'
            return current > best + self.threshold


In [45]:
# Определяем функцию потерь и оптимизатор градиентного спуска

loss_regression = nn.MSELoss()
opt_regression = torch.optim.Adam(model_regression.parameters(), lr=0.001)

In [46]:
reg_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                            opt_regression,       # оптимизатор
                            mode='min',           # 'max' или 'min'
                            factor=0.1,           # коэффициент, на который будет умножен lr
                            patience=5            # кол-во эпох без улучшения отслеживаемого параметра
                        )

earlystopping = EarlyStopping(mode='min', patience = 10)

In [47]:
# Проверка правильности построения модели

InpCheck = torch.rand([16, 1, 64, 64], dtype=torch.float32).to(device)

OutCheck = model_regression(InpCheck)
OutCheck.shape

torch.Size([16, 2])

## Тренировка модели

In [49]:
EPOCHS = 30
train_loss = []
train_acc = []
val_loss = []
val_acc = []
reg_lr_list = []
reg_best_loss = 100000
threshold = 0.0001
last_saved_reg_model = ''

In [50]:
# Цикл обучения
for epoch in range(EPOCHS):

    #режим тренировки
    model_regression.train()
    running_train_loss = []
    train_true_answers = 0
    train_loop = tqdm(bws_train_loader, leave=False)
    for x, targets in train_loop:
        x = x.to(device)               # (batch_size, 1, 64, 64)
        targets = targets.to(device)   # (batch_size, tensor[float, float])


        # прямой проход + расчет ошибки модели
        pred = model_regression(x)
        # pred - ответ модели,  target - правильные ответы -> подаем на вход ф-ции потерь
        loss = loss_regression(pred, targets)

        # обратный проход
        opt_regression.zero_grad() #обнуляем градиенты для каждого батча
        loss.backward() # рассчитываем градиентами для текущего батча

        # шаг оптимизации
        opt_regression.step() # Используем градиенты из шага выше для обновления весов

        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss)/len(running_train_loss)

        train_true_answers += (targets == torch.round(pred)).all(dim=1).sum().item()

        train_loop.set_description(f'Epoch: [{epoch+1}/{EPOCHS}], train_loss = {mean_train_loss:.4f}') 

    # Расчет метрик
    train_accuracy = train_true_answers/len(bws_train_data)
    
    # Сохранение значений функции потерь
    train_loss.append(mean_train_loss)
    train_acc.append(train_accuracy)
    

    #режим оценки
    model_regression.eval()
    with torch.no_grad():
        running_val_loss = []
        val_true_answers = 0
        val_loop = tqdm(bws_val_loader, leave=False)
        for x, targets in val_loop:
            x = x.to(device)               # (batch_size, 1, 64, 64)
            targets = targets.to(device)   # (batch_size, tensor[float, float])

            # прямой проход + расчет ошибки модели
            pred = model_regression(x)
            # pred - ответ модели,  target - правильные ответы -> подаем на вход ф-ции потерь
            loss = loss_regression(pred, targets)


            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss)/len(running_val_loss)

            val_loop.set_description(f'Epoch: [{epoch+1}/{EPOCHS}], train_loss = {mean_val_loss:.4f}') 

            val_true_answers += (targets == torch.round(pred)).all(dim=1).sum().item()

        # Расчет метрик
        val_accuracy = val_true_answers / len(bws_val_data)
        
        # Сохранение значений функции потерь
        val_loss.append(mean_val_loss)
        val_acc.append(val_accuracy)

    reg_lr_scheduler.step(mean_val_loss)
    lr = reg_lr_scheduler.get_last_lr()[0]
    reg_lr_list.append(lr)
    print(lr)

    print(f'Epoch: [{epoch+1}/{EPOCHS}], train_loss = {mean_train_loss:.4f}, train_acc={train_accuracy:.4f}, val_loss={mean_val_loss:.4f}, val_acc={val_accuracy:.4f}, lr={lr:.4f}')

    if reg_best_loss is None:
        reg_best_loss = mean_val_loss

    if mean_val_loss < reg_best_loss - reg_best_loss * threshold:
        reg_best_loss = mean_val_loss

        checkpoint = {
                        'state_model': model_regression.state_dict(),         # веса модели
                        'state_opt': opt_regression.state_dict(),             # состояние оптимизатора
                        'state_lr_scheduler': reg_lr_scheduler.state_dict(),  # состояние scheduler
                        'loss': {                                  # метрики потерь
                            'train_loss': train_loss,
                            'val_loss': val_loss,
                            'best_loss': reg_best_loss
                        },
                        'metric': {                                # метрики accuracy
                            'train_acc': train_acc,
                            'val_acc': val_acc,
                        },
                        'lr': reg_lr_list,                         # история learning rate
                        'epoch': {                                 # информация об эпохах
                            'EPOCHS': EPOCHS,
                            'save_epoch': epoch
                        }
                    }

        if os.path.exists(last_saved_reg_model):
            os.remove(last_saved_reg_model)
        last_saved_reg_model = f'models/bws_model_checkpoint_epoch_{epoch+1}.pt'
        torch.save(checkpoint, last_saved_reg_model)
        
        print(f' Эпоха {epoch+1}: функция потерь при валидации - {mean_val_loss:.4f}', end='\n\n')
        print(f' Модель: {last_saved_reg_model}')

    if earlystopping(mean_val_loss):
        print(f'\033[31mОбучение остановлено на {epoch + 1} эпохе.\033[0m')
        break

0.001
Epoch: [1/30], train_loss = 1.9030, train_acc=0.7652, val_loss=0.2879, val_acc=0.3506, lr=0.0010
 Эпоха 1: функция потерь при валидации - 0.2879

 Модель: models/bws_model_checkpoint_epoch_1.pt


Epoch: [2/30], train_loss = 0.1512:  26%|███████████▏                               | 567/2188 [01:52<04:56,  5.47it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

                                                                                                                       

0.001
Epoch: [3/30], train_loss = 0.0764, train_acc=0.8796, val_loss=0.0824, val_acc=0.8888, lr=0.0010
 Эпоха 3: функция потерь при валидации - 0.0824

 Модель: models/bws_model_checkpoint_epoch_3.pt


0.001
Epoch: [4/30], train_loss = 0.0690, train_acc=0.9163, val_loss=0.0305, val_acc=0.9948, lr=0.0010
 Эпоха 4: функция потерь при валидации - 0.0305

 Модель: models/bws_model_checkpoint_epoch_4.pt


0.001
Epoch: [5/30], train_loss = 0.0569, train_acc=0.9317, val_loss=0.0783, val_acc=0.8933, lr=0.0010


...

Epoch: [23/30], train_loss = 0.0175:  11%|████▍                                     | 233/2188 [00:38<05:29,  5.93it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

                                                                                                                       

0.001
Epoch: [24/30], train_loss = 0.0223, train_acc=0.9945, val_loss=0.0426, val_acc=0.9896, lr=0.0010


0.001
Epoch: [25/30], train_loss = 0.0220, train_acc=0.9957, val_loss=0.0180, val_acc=0.9979, lr=0.0010


Epoch: [26/30], train_loss = 0.0214:  96%|███████████████████████████████████████▏ | 2094/2188 [05:36<00:14,  6.45it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch: [30/30], train_loss = 0.0065:  57%|███████████████████████▍                 | 1252/2188 [03:47<03:27,  4.51it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

                                                                                                                       

0.0001
Epoch: [30/30], train_loss = 0.0064, train_acc=1.0000, val_loss=0.0106, val_acc=0.9999, lr=0.0001


## Проверка модели

In [51]:
model_regression.eval()
with torch.no_grad():
    running_test_loss = []
    test_true_answers = 0
    for x, targets in bws_test_loader:
        x = x.to(device)
        targets = targets.to(device)

        # прямой проход + расчет ошибки модели
        pred = model_regression(x)
        loss = loss_regression(pred, targets)
        
        running_test_loss.append(loss.item())
        mean_test_loss = sum(running_test_loss)/len(running_test_loss)
            
        test_true_answers += (targets == torch.round(pred)).all(dim=1).sum().item()

    # Расчет метрик
    running_test_acc = test_true_answers / len(bws_test_data)

print(f'test_loss = {mean_test_loss:.4f}, test_acc={running_test_acc:.4f}', end = '\n\n')

test_loss = 0.0107, test_acc=0.9998



## Вывод

**Результаты обучения модели регрессии на сгенерированном датасете:**

✅ **Модель успешно обучена за 30 эпох**, наилучший результат показала модель **bws_model_checkpoint_epoch_27**

📊 **Метрики качества:**
- **Точность на тестовой выборке:** 99.98% (`test_acc=0.9998`)
- **Ошибка на тестовой выборке:** 0.0107 (`test_loss=0.0107`)

🛠 **Параметры обучения:**
- **Архитектура:** Сверточная нейронная сеть (PyTorch)
- **Функция потерь:** `MSELoss`
- **Оптимизатор:** Adam с начальным learning rate 0.001
- **Планировщик LR:** `ReduceLROnPlateau` (уменьшает LR в 10 раз при отсутствии улучшений 5 эпох)
- **Ранняя остановка:** настройка на остановку на 10-й эпохе без улучшений, не было срабатываний

💾 **Лучшая модель сохранена на 27-й эпохе** с учетом:
- Весов модели и оптимизатора
- Гиперпараметров и истории learning rate
- Метрик обучения и валидации

**Вывод:** Модель безошибочно определила координаты центра белого квадрата более, чем в 99% тестовых данных. <br>
           Значительное улучшение качества модели было достигнуто при помощи динамического изменения learning rate.